In [2]:
import tensorflow as tf
print(tf.__version__)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

2.5.0


In [93]:
class MyModel():

    def __init__(self):
        '''
        Crear el modelos para segmentación
        '''
        self.fusion = {}
        self.input_rgb = tf.keras.layers.Input(shape=(640,480,3),dtype=tf.float32)
        self.input_depth = tf.keras.layers.Input(shape=(640,480,1),dtype=tf.float32)

    def rgbd_fusion(self,input_r,input_d,filters):
        r = tf.keras.layers.GlobalAveragePooling2D()(input_r)
        r = tf.keras.layers.Flatten()(r)
        r = tf.keras.layers.Dense(filters)(r)
        m_1 = tf.keras.layers.Multiply()([input_r,r])
        
        d = tf.keras.layers.GlobalAveragePooling2D()(input_d)
        d = tf.keras.layers.Flatten()(d)
        d = tf.keras.layers.Dense(filters)(d)
        m_2 = tf.keras.layers.Multiply()([input_d,d])
        
        return tf.keras.layers.Add()([m_1,m_2])

    def resnet_layer(self,input_res, filters):
        x = tf.keras.layers.BatchNormalization()(input_res)
        x = tf.keras.layers.ZeroPadding2D(padding=(1, 0))(x)
        x = tf.keras.layers.Conv2D(filters, (3, 1), strides=(2, 2))(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)
        
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ZeroPadding2D(padding=(0, 1))(x)
        x = tf.keras.layers.Conv2D(filters, (1, 3), strides=(2, 2))(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)

        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ZeroPadding2D(padding=(1, 0))(x)
        x = tf.keras.layers.Conv2D(filters, (3, 1), strides=(2, 2))(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)

        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ZeroPadding2D(padding=(0, 1))(x)
        x = tf.keras.layers.Conv2D(filters, (1, 3), strides=(2, 2))(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)


        # OTRA OPCION
        # x = BatchNormalization(name='bn_data', **no_scale_bn_params)(img_input)
        # x = ZeroPadding2D(padding=(3, 3))(x)
        # x = Conv2D(init_filters, (7, 7), strides=(2, 2), name='conv0', **conv_params)(x)
        # x = BatchNormalization(name='bn0', **bn_params)(x)
        # x = Activation('relu', name='relu0')(x)
        # x = ZeroPadding2D(padding=(1, 1))(x)
        # x = MaxPooling2D((3, 3), strides=(2, 2), padding='valid', name='pooling0')(x)
        
        return x

    def context_module(self,input_con):
        c_1 = tf.keras.layers.Conv2D(256, (1, 1))(input_con)
        # print(c_1.shape)
        # c_1 = tf.keras.layers.Flatten()(c_1)
        # c_1 = tf.keras.layers.Dense(256)(c_1)
        c_1 = tf.keras.layers.UpSampling2D(size=(1, 1), interpolation="nearest")(c_1)

        
        c_2 = tf.keras.layers.Conv2D(256, (1, 1))(input_con)
        # print(c_2.shape)
        # c_2 = tf.keras.layers.Flatten()(c_2)
        # c_2 = tf.keras.layers.Dense(256)(c_2)
        c_2 = tf.keras.layers.UpSampling2D(size=(1, 1), interpolation="nearest")(c_2)
        
        r = tf.keras.layers.Concatenate()([input_con, c_1, c_2])
        r = tf.keras.layers.Conv2D(1024,kernel_size=(1, 1),activation='relu')(r)
        return r
    
    def decode_layer(self,input_de):
        x = tf.keras.layers.Conv2D(filters, kernel_size=(3, 1), activation='relu', padding='same')(input_de)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv2D(filters, kernel_size=(1, 3), activation='relu', padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv2D(filters, kernel_size=(3, 1), activation='relu', padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv2D(filters, kernel_size=(1, 3), activation='relu', padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Add()([x, input_res])
        x = tf.keras.layers.Activation('relu')(x)
        return x

        
    def call(self):

        # Preparar la entrada de datos
        xr = tf.keras.layers.Conv2D(64,kernel_size=(7,7),strides=(2,2),padding='same',dilation_rate=(1,1),activation='relu')(self.input_rgb)
        xd = tf.keras.layers.Conv2D(64,kernel_size=(7,7),strides=(2,2),padding='same',dilation_rate=(1,1),activation='relu')(self.input_depth)

        # Fusion
        xr = self.rgbd_fusion(xr,xd,64)
        print('Fusion realizada')
        # Preparar para encoder
        xr = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(xr)
        xd = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(xd)

        # ENCODER

        i = 4 # las veces que se repite el resnet_layer y la fusion rgbd
        fact=1
        for j in range(4):
            
            n = 4 # las veces que se repite el resnet_layer
            for k in range(n):
                xr = self.resnet_layer(xr,fact*64)
                xd = self.resnet_layer(xd,fact*64)

            xr = self.rgbd_fusion(xr,xd,fact*64)
            p_fact = fact
            fact = p_fact + fact

            # Aqui guardamos la fusion realizada despues de la capa resnet. Esto lo utilizaremos para decodificar.
            self.fusion[j] = xr
        print('Encoder realizado')
        
        xr = self.context_module(xr)
        print('Context module realizado')


        print(xr.shape)
        xr = tf.keras.layers.Conv2D(512,kernel_size=(1,1),activation='relu')(xr)
        xr = tf.keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu')(xr)

        
        # DECODER



       



In [94]:
model = MyModel().call()
model.summary()

Fusion realizada
Encoder realizado
Context module realizado
(None, 1, 1, 1024)


ValueError: Negative dimension size caused by subtracting 3 from 1 for '{{node conv2d_2225/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](Placeholder, conv2d_2225/Conv2D/ReadVariableOp)' with input shapes: [?,1,1,512], [3,3,512,512].

In [ ]:
tf.keras.metrics.MeanIoU(num_classes, name=None, dtype=None)

In [5]:

print(resnet.get_layer('input_1'))

In [56]:
fact=1
for j in range(4):
    print(fact*64)
    p_fact = fact
    fact = p_fact + fact
    

1
3
5
7
9
